In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV # оптимальный подбор alpha
from sklearn.linear_model import Lasso, LassoCV # LassoCV - больше используется на практике
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics._scorer import _SCORERS

In [2]:
df = pd.read_csv('Advertising.csv')

In [3]:
df

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [4]:
X = df.drop('sales', axis=1)
y = df['sales']

In [5]:
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [6]:
poly_features = polynomial_converter.fit_transform(X) # на всём наборе

In [7]:
poly_features.shape

(200, 19)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [9]:
X_train.shape

(140, 19)

In [10]:
# масштабирование - только на обучающем наборе данных, чтобы не было утечки данных
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [11]:
X_train = scaler.transform(X_train)

In [12]:
X_test = scaler.transform(X_test)

In [13]:
X_train[0]

array([ 0.49300171, -0.33994238,  1.61586707,  0.28407363, -0.02568776,
        1.49677566, -0.59023161,  0.41659155,  1.6137853 ,  0.08057172,
       -0.05392229,  1.01524393, -0.36986163,  0.52457967,  1.48737034,
       -0.66096022, -0.16360242,  0.54694754,  1.37075536])

In [14]:
# L2 - регрессия
# снижает вероятность переобученности модели
# Для в RSS (остаточная сумма квадратов) добавляется штраф (penalty) , в котором содержатся квадраты коэффициентов
# Error = RSS + penalty -> min
# Кроме квадратов коэф. в штрафе находится параметр лямбда, который определяют силу штрафа или то, насколько учитываем штраф в нашей ошибке.
# в sklearn lambda == alpha

In [15]:
# для метрик кросс-валидации Sklearn использует объект scorer для выбора наилучшего значения параметра
# Правило - чем метрика больше, тем лучше
# для RMSE это не работает, поэтому для неё создали отрицательные значения

ridge_model = Ridge(alpha=10) # передали параметр штрафа

In [16]:
# дальше аналогично линейной регрессии
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [17]:
test_predictions = ridge_model.predict(X_test)

In [18]:
MAE = mean_absolute_error(y_test, test_predictions)
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [19]:
MAE

0.5774404204714166

In [20]:
RMSE

0.8946386461319648

In [21]:
# хочется узнать, какой aplha оптимальный
# из-за того, что это частая задача, существует целая библиотека from sklearn.linear_model import RidgeCV. CV - кросс-валидация

In [22]:
# пробуем найти оптимальное значение для alpha

ridge_cv_model = RidgeCV(alphas=(0.01, 0.1, 1, 10),scoring='neg_mean_absolute_error') # scoring - по какой метрике будет определяться результат, cv - число разбиения набора данных
# cv = None => объем данных маленький => Leave-One-out - работа по одной строке
# Если данных много, то лучше cv = 10
# основное - задать alpha

In [23]:
ridge_cv_model.fit(X_train, y_train) # используем только обучающий набор данных для настройки гиперпараметров

RidgeCV(alphas=(0.01, 0.1, 1, 10), scoring='neg_mean_absolute_error')

In [24]:
ridge_cv_model.alpha_ # узнаем лучшее значение по alpha_
# можем дополнительно написать alpha в окрестности ответа, чтобы точнее найти лучший коэффициент

0.01

In [25]:
_SCORERS.keys() # список различных метрик

dict_keys(['explained_variance', 'r2', 'neg_max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_root_mean_squared_log_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'd2_absolute_error_score', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'positive_likelihood_ratio', 'neg_negative_likelihood_ratio', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'reca

In [26]:
test_predictions = ridge_cv_model.predict(X_test)

In [27]:
MAE = mean_absolute_error(y_test, test_predictions)
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [28]:
# X_test, Y_test в данном случае - это hold_out_test_set

In [29]:
MAE

0.4101873183288653

In [30]:
RMSE

0.5829610859820523

In [31]:
ridge_cv_model.coef_

array([ 6.9038259 ,  0.49047734,  0.29464737, -9.7103904 ,  5.06456503,
       -1.66613758, -1.30427348,  0.76860722,  0.35722042,  4.58992377,
       -1.52624141,  1.42375451,  0.43356846, -0.31710017,  0.03883517,
        0.71412093, -0.3097189 , -0.21689709, -0.24610412])

In [32]:
ridge_cv_model.best_score_

-0.36285148629181346

In [33]:
# L1 - регуляризация - Lasso - оператор наименьшей абсолютной ошибки и отбора
# задать alpha можно задать через диапазон либо через eps = alpha_min / alpha_max

In [42]:
lasso_cv_model = LassoCV(eps=0.1, n_alphas=100) #n_alphas позволяет выбрать, сколько значений alpha будет рассмотрено при поиске оптимального параметра регуляризации

In [43]:
lasso_cv_model.fit(X_train, y_train)
#ConvergenceWarning. Не достигнута сходимость по параметру alpha при обучении => max_iter не достаточно. По умолчанию max_iter = 1000
# ещё можно увеличить eps, чтобы не было ошибки

LassoCV(eps=0.1)

In [45]:
test_predictions = lasso_cv_model.predict(X_test)

In [44]:
lasso_cv_model.alpha_

0.4943070909225831

In [46]:
MAE = mean_absolute_error(y_test, test_predictions)
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [47]:
MAE

0.6541723161252868

In [48]:
RMSE

1.1308001022762548

In [50]:
# По метрикам хуже, чем Ridge регрессия, но есть выигрыш в коэффициентах

In [51]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])